In [2]:
def tokenize_reply(reply):
    # Use blank to tokenize
    tokens = reply.split()

    if len(tokens) == 1:
        # if the reply is only one word, use it as name
        return tokens[0]
    else:
        # find key word and use the word after it as name
        keywords = ['me', 'is', "it's", 'am', "I'm"]
        for i, token in enumerate(tokens):
            if token.lower() in keywords and i < len(tokens) - 1:
                return tokens[i+1]
    return None

def lipstick_assistant():
    print("Hello! I'm Sephora's virtual lipstick assistant, you can call me Sophia.")
    customer_name = input("What shall I call you? ")
    customer_name = tokenize_reply(customer_name)
    input(f"Hello, {customer_name}! How's your day going so far?")

lipstick_assistant()


Hello! I'm Sephora's virtual lipstick assistant, you can call me Sophia.
What shall I call you? e
Hello, e! How's your day going so far?g'


In [19]:
import requests

url = "https://sephora14.p.rapidapi.com/searchByKeyword"

querystring = {"search":"lipstick","sortBy":"NEW","page":"1"}

headers = {
	"X-RapidAPI-Key": "64bf397be2msh4f1d00e055699dep148644jsnecf43e9edee8",
	"X-RapidAPI-Host": "sephora14.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())


{'products': [{'brandName': 'Charlotte Tilbury', 'currentSku': {'imageAltText': 'Charlotte Tilbury - K.I.S.S.I.N.G Satin Shine Lipstick', 'isAppExclusive': False, 'isBI': False, 'isBest': False, 'isFirstAccess': False, 'isLimitedEdition': False, 'isLimitedTimeOffer': False, 'isNatural': False, 'isNew': True, 'isOnlineOnly': False, 'isOrganic': False, 'isSephoraExclusive': False, 'listPrice': '$35.00 - $38.00', 'skuId': '2765600', 'skuType': 'Standard'}, 'displayName': 'K.I.S.S.I.N.G Satin Shine Lipstick', 'heroImage': 'https://www.sephora.com/productimages/sku/s2765600-main-zoom.jpg?imwidth=270', 'image135': 'https://www.sephora.com/productimages/sku/s2765600-main-zoom.jpg?imwidth=270', 'image250': 'https://www.sephora.com/productimages/sku/s2765600-main-zoom.jpg?imwidth=270', 'image450': 'https://www.sephora.com/productimages/sku/s2765600-main-zoom.jpg?imwidth=270', 'altImage': 'https://www.sephora.com/productimages/sku/s2765600-av-2-zoom.jpg?imwidth=270', 'moreColors': 21, 'onSaleDat

In [20]:
import re

def extract_textures(product_name, texture_keywords):
    # Check for valid inputs
    if not product_name or not texture_keywords:
        return []

    # Create a regular expression pattern that matches any of the texture keywords
    pattern = re.compile(r'(' + '|'.join([re.escape(keyword) for keyword in texture_keywords]) + r')', re.IGNORECASE)

    # Initialize an empty set to store unique texture matches
    textures = set()

    # Find all matches of texture keywords in the product name
    matches = pattern.findall(product_name)
    textures.update([match.lower() for match in matches])  # Normalize to lower case for consistency

    return list(textures)

In [21]:
texture_map = {
    'moist': ['Hydrating','Creamy', 'Sheer', 'Glossy' ,'Natural', 'Velvet','Shine'],
    'dry': ['Matte', 'Velvet'],
    'neutral': [ 'Creamy', 'Sheer', 'Natural', 'Velvet', 'Satin','Silk','Smoothing']
}

In [22]:
conceal_map = {
    'strong': ['Matte', 'Sheer', 'Velvet'],
    'light': ['Hydrating', 'Sheer', 'Glossy', 'Natural'],
    'neutral': ['Matte', 'Velvet', 'Satin', 'Creamy', 'Hydrating', 'Sheer', 'Glossy', 'Natural']
}

In [23]:
import re
def extract_brand_texture(product_name, texture_keywords):
    parts = product_name.split(' - ')
    brand = parts[0].strip()
    textures = []

    pattern = re.compile(r'\b(' + '|'.join(texture_keywords) + r')\b', re.IGNORECASE)

    for part in parts[1:]:
        matches = pattern.findall(part)
        textures.extend(matches)

    return brand, list(set(textures))


def extract_from_sentence(sentence, options, feedback={}):
    for option, keywords in options.items():
        for keyword in keywords:
            if re.search(r'\b{}\b'.format(re.escape(keyword)), sentence, re.IGNORECASE):
                print(feedback.get(option, ""))
                return option
    print(feedback.get('default', ""))
    return 'Not specified'

In [33]:

def get_texture_preference():
    texture_options = {
        'dry': ['matte', 'dry'],
        'moist': ['hydrating', 'moist', 'moisture'],
        'neutral': ['balanced', 'neutral', 'natural']
    }
    texture_feedback = {
        'dry': "Our matte lipsticks offer a dry texture without the parched feeling.",
        'moist': "Moisture is key! Let's get you a lipstick that offers a moist texture for that all-day hydration.",
        'neutral': "Balance is beautiful. A natural texture lipstick will give you the best of both worlds.",
        'default': "Let's aim for a balanced finish with a neutral texture."
    }
    response = input("Are you looking for a lipstick with a dry, moist, or natural finish? ")
    return extract_from_sentence(response, texture_options, texture_feedback)

def get_conceal_preference():
    conceal_options = {
        'strong': ['full coverage', 'opaque', 'strong'],
        'light': ['sheer', 'tint', 'light'],
        'neutral': ['natural', 'neutral']
    }
    conceal_feedback = {
        'strong': "Full coverage coming up! You'll get all the attention with our richly pigmented shades.",
        'light': "A hint of color, I see. We'll keep it sheer.",
        'neutral': "Natural it is. It's all about enhancing your inherent beauty subtly.",
        'default': "Let's go with a natural look, offering a subtle enhancement."
    }
    response = input("Now, shall we go for a lipstick that provides strong coverage, or would you like to keep things light? ")
    return extract_from_sentence(response, conceal_options, conceal_feedback)


def get_price_limit():
    response = input("To finish off, what price range are we considering for your ideal lipstick? ")
    price_search = re.search(r'\$?\d+', response)
    price = '$' + price_search.group(0) if price_search else 'Not specified'
    print(f"Got it, we'll look for options around {price}.")
    return price

In [34]:
user_preferences = {}

In [35]:
def pipeline(products, filters):
    for f in filters:
        #Apply filters
        p = f(products)
        if len(p) == 0:
            return (products, True)
        else:
            products = p
    return (products, False)

In [36]:
def textureFilter(products):
    if not 'texture' in user_preferences:
        return products
    res = []
    for product in products:
        brand, textures = extract_brand_texture(product['currentSku']['imageAltText'], texture_map[user_preferences['texture']])
        if not len(textures) == 0:
            res.append(product)
    return res

In [37]:
def concealFilter(products):
    if not 'conceal' in user_preferences:
        return products
    res = []
    for product in products:
        brand, conceals = extract_brand_texture(product['currentSku']['imageAltText'], conceal_map[user_preferences['conceal']])
        if not len(conceals) == 0:
            res.append(product)
    return res

In [38]:
def priceFilter(products):
    if not 'price' in user_preferences:
        return products
    res = []
    for product in products:
        listPrice = product['currentSku']['listPrice']
        if '-' in listPrice:
            listPrice = listPrice.split('-')[1].strip()
        if float(listPrice[1:]) <= float(user_preferences['price'][1:]):
            res.append(product)
    return res

In [39]:
def main():
    print("Good to know! Are you ready for a journey of beauty adventure? Let's find your ideal match.")
    global user_preferences
    user_preferences = {
        'texture': get_texture_preference(),
        'conceal': get_conceal_preference(),
        'price': get_price_limit(),
    }

    print("\nBased on your preferences, here's what you indicated:")
    for pref, value in user_preferences.items():
        print(f"{pref.capitalize()}: {value}")

In [40]:
main()

Good to know! Are you ready for a journey of beauty adventure? Let's find your ideal match.
Are you looking for a lipstick with a dry, moist, or natural finish? dry
Our matte lipsticks offer a dry texture without the parched feeling.
Now, shall we go for a lipstick that provides strong coverage, or would you like to keep things light? light
A hint of color, I see. We'll keep it sheer.
To finish off, what price range are we considering for your ideal lipstick? 40
Got it, we'll look for options around $40.

Based on your preferences, here's what you indicated:
Texture: dry
Conceal: light
Price: $40


In [41]:
filters = [textureFilter, concealFilter, priceFilter]
final_products, truncated = pipeline(response.json()['products'], filters)

In [42]:
final_products.sort(key=lambda x: float(x['rating']), reverse=True)
first_five = final_products[:5]

In [43]:
print("Top 5 Products:")
for product in first_five:
    print(f"Name: {product['brandName'],product['productName']}, Rating: {product.get('rating', 'No rating')},Image: {product.get('heroImage')}, link: {product.get('targetUrl')}")

Top 5 Products:
Name: ('Charlotte Tilbury', 'Matte Revolution Hydrating Lipstick'), Rating: 4.2417,Image: https://www.sephora.com/productimages/sku/s2736098-main-zoom.jpg?imwidth=270, link: https://www.sephora.com/product/matte-revolution-lipstick-P433530?skuId=2736098


In [45]:
import requests

url = "https://sephora14.p.rapidapi.com/searchByKeyword"

headers = {
    "X-RapidAPI-Key": "64bf397be2msh4f1d00e055699dep148644jsnecf43e9edee8",
    "X-RapidAPI-Host": "sephora14.p.rapidapi.com"
}

In [46]:
specialEffects = []

In [52]:
def get_special_effect_preference():
    special_effect_options = {
        'yes': ['yes','pigment', 'plumping', 'long-lasting', 'metallic', 'shimmer'],
        'no': ['no', 'none']
    }
    special_effect_feedback = {
        'yes': "Great choice! Adding some flair to your look.",
        'no': "Keeping it classic, I see. No special effects needed.",
        'default': "We can decide on the special effects later."
    }
    response = input("To add some flair, are you interested in any special effects, like a high pigment punch, a plumping effect, something long-lasting, or perhaps a touch of metallic shimmer? ")
    return 'yes' if 'yes' in extract_from_sentence(response, special_effect_options, special_effect_feedback).lower() else 'no'

In [58]:
import datetime

# Mapping of weekdays to special effects and their associated chatbot messages
effect_and_message = {
    0: ('pigment', "Happy Monday! Ready to start the week with a splash of color? Today's special is all about high-pigment lipsticks. They’re perfect for making that Monday meeting a bit more memorable. Shall I show you our most popular shades?"),
    1: ('plumping', "It’s Tuesday, and we’re all about that plump! Our lip plumping formulas are on special today. They give you that enviable full-lip look with each application. Interested in giving your lips a little Tuesday treat?"),
    2: ('long-lasting', "Hello, Wednesday warrior! Our long-lasting lipsticks are on the frontline today. From your first coffee to your last call, we’ve got you covered. How about we find a color that sticks with you all day?"),
    3: ('metallic', "Thrilling Thursday calls for a touch of metallic magic! Today we’re featuring lipsticks with a metallic finish to add some sparkle to your week. Ready to shine like the star you are?"),
    4: ('SPF', "TGIF! Our Friday feature is SPF-infused lipsticks, because your lips need sun protection too. Heading out for a sunny weekend adventure? Let’s keep those lips protected and pretty."),
    5: ('natural', "Saturday’s style is natural and effortless. Today, we’re showcasing lipsticks that enhance your natural beauty. Perfect for a casual brunch or a relaxing day in. Shall we pick a shade that’s as easygoing as your Saturday?"),
    6: ('acid', "Sundays are for self-care, and what's better than a rejuvenating acid-infused lipstick? They gently exfoliate, revealing smooth, soft lips for the week ahead. Would you like to end your week with a lip revival?")
}

def get_daily_special_effect():
    today = datetime.datetime.now().weekday()  # Monday is 0, Sunday is 6
    effect, message = effect_and_message[today]
    print(message)  # Print the message for today's special effect
    return effect


def specialEffectFilter(products):
        res = []
        for product in products:
            brand, effects = extract_brand_texture(product['currentSku']['imageAltText'], specialEffects)
            if not len(effects) == 0:
                res.append(product)
        return res

def main():
    global specialEffects
    specialEffects = [get_daily_special_effect()]

    # Apply the pipeline with the configured filter
    special_effect_products, truncated = pipeline(response.json()['products'], [specialEffectFilter])

    # Output or process the filtered products as needed
    for product in special_effect_products:
        print(f"Name: {product['brandName'],product['productName']}, Rating: {product.get('rating', 'No rating')},Image: {product.get('heroImage')}, link: {product.get('targetUrl')}")  # Adjust this to match how you want to display the products

main()

Happy Monday! Ready to start the week with a splash of color? Today's special is all about high-pigment lipsticks. They’re perfect for making that Monday meeting a bit more memorable. Shall I show you our most popular shades?
Name: ('Danessa Myricks Beauty', 'Colorfix - Multi-Use Eye, Cheek & Lip Waterproof Liquid Pigment'), Rating: 4.3916,Image: https://www.sephora.com/productimages/sku/s2774354-main-zoom.jpg?imwidth=270, link: https://www.sephora.com/product/danessa-myricks-colorfix-24-hour-cream-color-P468353?skuId=2774354
Name: ('HAUS LABS BY LADY GAGA', 'Hy-Power Eye, Cheek & Lip Pigment Paint'), Rating: 4.2383,Image: https://www.sephora.com/productimages/sku/s2755957-main-zoom.jpg?imwidth=270&pb=clean-at-sephora, link: https://www.sephora.com/product/hy-power-eye-cheek-lip-pigment-paint-P500298?skuId=2755957
Name: ('Yves Saint Laurent', 'The Bold High Pigment Lipstick'), Rating: 4.5253,Image: https://www.sephora.com/productimages/sku/s2662294-main-zoom.jpg?imwidth=270, link: http